In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

test.csv
train.csv



In [14]:
Train_Data=pd.read_csv("../input/train.csv")
Test_Data=pd.read_csv("../input/test.csv")

<h1>Exploratory Analysis </h2>

In [ ]:
len(Train_Data)

In [ ]:
len(Test_Data)

Let us train and test our model using the Training Data and perform final validation on the given test data. 

In [ ]:
Train_Data.sample(frac=0.1).head(n=5)

In [ ]:
Train_Data.describe()

The below lists the variables in the dataset. 

In [ ]:
list(Train_Data.columns.values)

<h1>Pre-Processing </h1>

The activities are in the form of text which cannot be used directly as input features for machine learning algorithms So ,  convert them to categorical numerical data.

In [15]:
#Split the Data for machine learning algorithms

def Featurize_Outcome(Data):
    
        '''The activity variable is in the form of strings , which cannot be directly given as input to the algorithm , 
        for the purpose we will convert them to numerical features'''
        
        Labels=[]
        Key={}

        Prev=None
        Index=0

        for i in Data:
        
                    if(i in Key):

                         Labels.append(Key[i])
        
                    else:
        
                         Key[i]=Index
                         Labels.append(Key[i])
                         Index+=1
                    
        return(Labels)
    
Train_Data_X=Train_Data
Train_Data_Y=Train_Data['Activity']
Train_Data_Y=Featurize_Outcome(Train_Data_Y)
Train_Data_X.drop('Activity',axis=1,inplace=True)

Test_Data_X=Test_Data
Test_Data_Y=Test_Data['Activity']
Test_Data_Y=Featurize_Outcome(Test_Data_Y)
Test_Data_X.drop('Activity',axis=1,inplace=True)


<h2>Learning Algorithms (Original Dataset)</h2>

In [25]:
#Importing required libraries for developing machine learning models

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
from sklearn.ensemble import RandomForestClassifier
Rf=RandomForestClassifier()
print("Accuracy: ",cross_val_score(Rf,Train_Data_X,Train_Data_Y, cv=5).mean())

In [ ]:
gNB=GaussianNB()
scores=cross_val_score(gNB,Train_Data_X,Train_Data_Y, cv=5)
print("Accuracy: ",scores.mean())

In [ ]:
cl=tree.DecisionTreeClassifier()
print("Accuracy: ",cross_val_score(cl,Train_Data_X,Train_Data_Y,cv=5).mean())

In [ ]:
clf=svm.SVC(kernel='rbf')
print("Accuracy: ",cross_val_score(clf,Train_Data_X,Train_Data_Y, cv=5).mean())

In [ ]:
clf=svm.SVC(kernel='linear')
print("Accuracy: ",cross_val_score(clf,Train_Data_X,Train_Data_Y, cv=5).mean())

In [ ]:
scores=[]
for i in range(1,31):
    neigh=KNeighborsClassifier(n_neighbors=i)
    scores.append(cross_val_score(neigh,Train_Data_X,Train_Data_Y,cv=5).mean())
    
max_a=0
k_max=0

for i in range(0,30):
    
    if(scores[i]>=max_a):
        
        max_a=scores[i]
        
        if(i>k_max):
                
            k_max=i
        
print("K is maximum in Knn for ",k_max," with a accuracy of ",max_a) 

In [ ]:
AB=AdaBoostClassifier()
print("Accuracy: ",cross_val_score(AB,Train_Data_X,Train_Data_Y, cv=5).mean())

<h2>Learning Algorithms (Dataset with Feature Selection Applied)</h2>

There are 563 columns of data which could be time consuming to train and could possibly lead to overfitting. It could possibly help to use Dimensionality reduction and feature selection algorithms.  Lets apply various feature elimination methods to Support Vector Machine with Linear Kernel.

<h2>Feature Elimination Based on Variance</h2>

In [ ]:
#Import the requird library required for feature elimination based on variance. 
from sklearn.feature_selection import VarianceThreshold

In [ ]:
sel=VarianceThreshold(.9 * (1 - .9))
FS_Train_Data_X=sel.fit_transform(Train_Data_X)

In [ ]:
FS_Train_Data_X.shape

The number of relevant features are reduced from 562 to 212 upon eliminating features based on variance. 

In [ ]:
clf=svm.SVC(kernel='linear')
print("Accuracy: ",cross_val_score(clf,FS_Train_Data_X,Train_Data_Y, cv=5).mean())

From the above it could be noted that the there is a fall in accuracy upon feature elimination based on low variance. Lets try recursive feature elimination which involves a exhaustive search of best features using recursion.

<h2>Recursive Feature Elimination</h2>

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [ ]:
clf=svm.SVC(kernel='linear')
rfecv = RFECV(estimator=clf, step=1, cv=StratifiedKFold(2),
              scoring='accuracy')
rfecv.fit(Train_Data_X,Train_Data_Y)

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
print(max(rfecv.grid_scores_))

Although eliminating features led to just a 1% improvement in accuracy , it could lead to a much simpler model making it less prone to overfitting. Let us evaluate the model on our final validation test set. 

In [ ]:
print("Accuracy: ",cross_val_score(rfecv,Test_Data_X,Test_Data_Y,cv=5).mean())

From the above it could be noted that the algorithm peforms well even on the test set. 

<h1>PCA Feature Representation</h1>

In [65]:
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
n_components=175

In [60]:
pca_featurize = PCA(n_components=n_components, svd_solver='randomized',whiten=True).fit(Train_Data_X)

In [61]:
train_x_pca=pca_featurize.transform(Train_Data_X)
test_x_pca=pca_featurize.transform(Test_Data_X)

In [68]:
clf=svm.SVC(kernel='linear')
clf.fit(train_x_pca,Train_Data_Y)
print("Accuracy: ",cross_val_score(clf,train_x_pca,Train_Data_Y, cv=5).mean())

Accuracy:  0.935144187030138


In [71]:
print(classification_report(Test_Data_Y,clf.predict(test_x_pca)))

             precision    recall  f1-score   support

          0       0.89      0.97      0.93       532
          1       0.97      0.86      0.91       491
          2       1.00      1.00      1.00       537
          3       0.95      0.99      0.97       496
          4       1.00      0.96      0.98       420
          5       0.97      0.96      0.96       471

avg / total       0.96      0.96      0.96      2947



<h1>Autoencoder Feature Representation</h1>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
X=tf.placeholder(tf.float32,[None,562])

In [ ]:
#Encoder

w1=tf.Variable(tf.truncated_normal([562,400],stddev=0.1))
b1=tf.Variable(tf.constant(0.1,shape=[400]))

l1=tf.nn.sigmoid(tf.add(tf.matmul(X,w1),b1))

w2=tf.Variable(tf.truncated_normal(shape=[400,300],stddev=0.1))
b2=tf.Variable(tf.constant(0.1,shape=[300]))

l2=tf.nn.sigmoid(tf.add(tf.matmul(l1,w2),b2))

w3=tf.Variable(tf.truncated_normal(shape=[300,200],stddev=0.1))
b3=tf.Variable(tf.constant(0.1,shape=[200]))

Encoded_Layer=tf.nn.relu(tf.add(tf.matmul(l2,w3),b3))

In [ ]:
#Decoder

w4=tf.Variable(tf.truncated_normal(shape=[200,300],stddev=0.1))
b4=tf.Variable(tf.constant(0.1,shape=[300]))

l4=tf.nn.sigmoid(tf.add(tf.matmul(Encoded_Layer,w4),b4))

w5=tf.Variable(tf.truncated_normal(shape=[300,400],stddev=0.1))
b5=tf.Variable(tf.constant(0.1,shape=[400]))

l5=tf.nn.sigmoid(tf.add(tf.matmul(l4,w5),b5))

w6=tf.Variable(tf.truncated_normal(shape=[400,562],stddev=0.1))
b6=tf.Variable(tf.constant(0.1,shape=[562]))

#Output from decoder

y=tf.add(tf.matmul(l5,w6),b6)

In [ ]:
#Loss Function
loss=tf.reduce_sum(tf.pow(y-X,2))
train_step=tf.train.AdamOptimizer(1e-5).minimize(loss)

In [ ]:
#Initialize and Run Tensorflow Session

Loss=[]

epoch_num=100

batch_no=0
batch_size=64

max_epoch=10
iters_num=100000

loss_history=[]

train_set_size=len(Train_Data_X)

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for iters in range(0,iters_num):
        
           batch = Train_Data_X[batch_no:batch_no+batch_size]
           sess.run(train_step,feed_dict={X:batch})
        
           if((batch_no+batch_size)<train_set_size):
                    
                    batch_no+=batch_size
                    
           else:
            
                    batch_no=0
                    loss_no=sess.run(loss,feed_dict={X:Train_Data_X})
                    print(loss_no)
                    loss_history.append(loss_no)
                    plt.plot(loss_history)
                    plt.show()
                
            